In [1]:
import pandas as pd

In [ ]:
excel_file = 'INSERT FILE PATH HERE'
# excel_file = '/home/.../Downloads/SampleOverview.-.Kopie.xlsx'

In [3]:
df = pd.read_excel(excel_file)
# Remove trailing rows where all values are NaN or NaT
df_trimmed = df.loc[:df.dropna(how='all').index[-1]]
# Fill NaN values with 'x'
df_trimmed['Log Name'] = df_trimmed['Log Name'].fillna('x')
# Split the DataFrame
df_log_name_empty_or_x = df_trimmed[df_trimmed['Log Name'].isin(['', 'x'])]
df_sub_ref_nan = df_trimmed[df_trimmed['Sub ref'].isna()]
df_rest = df_trimmed[~df_trimmed['Log Name'].isin(['', 'x']) & df_trimmed['Sub ref'].notna()]

# Save the unprocessed DataFrames to Excel files
df_log_name_empty_or_x.to_excel('df_log_name_empty_or_x.xlsx', index=False)
df_sub_ref_nan.to_excel('df_sub_ref_nan.xlsx', index=False)

print(f'Able to process {len(df_rest)}/{len(df_trimmed)} rows')

Able to process 338/358 rows


/tmp/ipykernel_49603/554015205.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trimmed['Log Name'] = df_trimmed['Log Name'].fillna('x')


In [4]:
# Add a new column with '.elog' replaced by '.dlog'
df_rest['dlog'] = df_rest['Log Name'].str.replace('.elog', '.dlog')

/tmp/ipykernel_49603/2114905016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rest['dlog'] = df_rest['Log Name'].str.replace('.elog', '.dlog')


In [22]:
def to_number_or_none(value, factor=1.0):
    try:
        if pd.isna(value):
            return None
        return float(value) * factor
    except ValueError:
        return None

In [ ]:
# List to store JSON objects
json_list = []

# Iterate over the DataFrame
for index, row in df_rest.iterrows():
    targets = []
    rms = []
    thicknesses = []
    distances = []
    if row['Target ref'] != '-':
        targets.append(row['Target ref'].replace('"', ''))
        thicknesses.append(to_number_or_none(row['d(SAO) \n[nm]']))
        distances.append(to_number_or_none(row['Target distance1\n[mm]'], factor=1e-3))
    if row['Target2 ref'] != '-':
        targets.append(row['Target2 ref'].replace('"', ''))
        thicknesses.append(to_number_or_none(row['d(CL) \n[nm]']))
        distances.append(to_number_or_none(row['Target distance2\n[mm]'], factor=1e-3))
    if len(targets) == 2:
        rms.append(None)
    rms.append(to_number_or_none(row['RMS\n[pm]']))
    json_obj = {
        "data": {
            "m_def": "nomad_ikz_plugin.pld.schema.IKZPulsedLaserDeposition",
            "name": row['ID\n'],
            "substrate": row['Sub ref'].replace('"', ''),
            "targets": targets,
            "data_log": row['dlog'],
            "recipe_log": row['Log Name'],
            "target_distances": distances,
            "_rms": rms,
            "_thicknesses": thicknesses,
        }
    }
    json_list.append(json_obj)

In [30]:
import json
import os
import zipfile

# Create JSON files and zip them together
with zipfile.ZipFile('json_files.zip', 'w') as zipf:
    for json_obj in json_list:
        file_name = f'{json_obj["data"]["name"]}.archive.json'
        with open(file_name, 'w') as json_file:
            json.dump(json_obj, json_file, indent=4)
        zipf.write(file_name)
        os.remove(file_name)